In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import re
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
driver = webdriver.Chrome(executable_path="/Users/jungjihoon/Downloads/chromedriver")
driver.maximize_window()
url = 'https://www.jejupassrent.com/home/main/main.do?&ACE_REF=adwords_g&ACE_KW=&gclid=Cj0KCQjw9_mDBhCGARIsAN3PaFO-J4-X9X-WGcBtvzWtdru1Hrevg3PpJo-57kW0ZEscx5bqs2-GJtAaAg7wEALw_wcB'
driver.get(url=url)

In [3]:
# 리스트 생성
name =[]
car = []
cpct = []
fuel = []
entp = []
age = []
ins_type = []
review = []
reserv = []
star = []
dprice = []
rdate =[]
lrc = []
df_option = pd.DataFrame(columns=["연식", "에어백", "후방센서", "후방카메라", "블랙박스", "4륜구동", "금연차량", "스팀소독", "네비게이션",
                               "스마트키", "열선시트", "열선핸들", "통풍시트", "썬루프", "블루투스", "낚시용품허용"])

In [4]:
soup = BeautifulSoup(driver.page_source, 'lxml')
interval = 20
inte = 12

In [5]:
start_day_list = []
times = datetime.date(2021, 8, 1)
for i in range(30):
    start_day_list.append(times.strftime('%Y%m%d'))
    times += datetime.timedelta(days=1)
    
end_day_list = []
times = datetime.date(2021, 8, 2)
for i in range(30):
    end_day_list.append(times.strftime('%Y%m%d'))
    times += datetime.timedelta(days=1)

In [6]:
for start_day, end_day in zip(start_day_list, end_day_list):

    # 날짜 클릭
    driver.find_element_by_class_name('clickTimesDate').click()
    driver.execute_script("tdClick(" + f"{start_day}" + ",this)")
    driver.execute_script("tdClick(" + f"{end_day}" + ",this)")
    driver.execute_script("fncApplyCalendarPanel();")
    driver.execute_script("fncApplyCalendarPanel();")
    driver.execute_script("fncSearchRentCar();")
    time.sleep(interval)

    # Bs4로 titles에 컬럼 담기
    soup = BeautifulSoup(driver.page_source, 'lxml')
    titles = soup.find_all('div', attrs={'class':'title'})
    for title in titles:
        rdate.append(start_day)
        name.append(title.find('h3').get_text())
        car.append(title.find_all('li')[0].string)
        cpct.append(title.find_all('li')[1].string)
        fuel.append(title.find_all('li')[2].string)

    time.sleep(inte)

    # prices에 있는 컬럼 담기
    soup = BeautifulSoup(driver.page_source, 'lxml')
    prices = soup.find_all('table', 'table table-price')
    for price in prices: 
        a = re.findall('\w',price.find('tr','selected').find_all('td')[1].get_text())
        b = "".join(a)
        entp.append(re.sub('[친절1위,편의1위,청결1위]',"",str(b)))
        age.append(price.find('tr', 'selected').find_all('td')[2].get_text())
        ins_type.append(price.find('tr', 'selected').find_all('td')[3].get_text())
        review.append(price.find('tr', 'selected').find_all('td')[4].get_text())
        reserv.append(price.find('tr', 'selected').find_all('td')[5].get_text())
        star.append(price.find('tr', 'selected').find_all('td')[6].get_text())
        dprice.append(price.find('tr', 'selected').find_all('td')[7].get_text())

    time.sleep(inte)
    
    # rentcar 취급 개수
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    ls_rentcar_count = []
    #rentcar 취급개수 list
    
    rentcar_counts = soup.select("ul.search-car-result-list")[0].select('li.panel > div.right > div.table-wrap > table > tbody > tr')
    rentcar_counts.append(rentcar_counts[0])
    for idx, rentcar_count in enumerate(rentcar_counts):
        if '최저가순' in rentcar_count.get_text():
            ls_rentcar_count.append(idx)

    for i in range(0,len(ls_rentcar_count)-1):
        lrc.append(ls_rentcar_count[i+1] - ls_rentcar_count[i] -1)
    
    time.sleep(inte)

    #옵션 dataframe 만들기
    ls_test = []
    options = soup.find_all('ul','list-option')


    for option in options:
        test = option.find_all('span',"on")
        for a in test:
            ls_test.append(a.string)
            ls_sum = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

            for v in ls_test:
                if "~" in v:
                    ls_sum += np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
                elif v == "에어백":
                    ls_sum += np.array([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
                elif v == "후방센서":
                    ls_sum += np.array([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
                elif v == "후방카메라":
                    ls_sum += np.array([0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
                elif v == "블랙박스":
                    ls_sum += np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0])
                elif v == "4륜구동":
                    ls_sum += np.array([0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0])
                elif v == "금연차량":
                    ls_sum += np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0])
                elif v == "스팀소독":
                    ls_sum += np.array([0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0])
                elif v == "네비게이션":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0])
                elif v == "스마트키":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0])
                elif v == "열선시트":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0])
                elif v == "열선핸들":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0])
                elif v == "통풍시트":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0])
                elif v == "썬루프":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0])
                elif v == "블루투스":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0])
                elif v == "낚시용품허용":
                    ls_sum += np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])



        ls_sum = list(ls_sum)

        ls_test = []

        df_option = df_option.append(pd.Series(ls_sum, index=df_option.columns), ignore_index=True)



    time.sleep(inte)


    # 뒤로가기
    driver.back()
    time.sleep(inte)



In [7]:

rent = pd.DataFrame({'Car_name':name, 'Car_type':car,'Capacity':cpct, 'Fuel_type':fuel,
                     'EnterP': entp,'Age':age, 'InsuranceType':ins_type,'Review':review,
                     'Resevation':reserv,'Star':star,"Date":rdate,'Price':dprice, 'rentcar_company_count':lrc})
rent

,Car_name,Car_type,Capacity,Fuel_type,EnterP,Age,InsuranceType,Review,Resevation,Star,Date,Price,rentcar_company_count
0,올 뉴 모닝(17년~)(휘),경형,5인승,휘발유,특별한렌트카,만 26세,완전자차,리뷰 58,누적예약 623,4.2,20210801,"93,100원",3
1,엑센트(휘),소형,5인승,휘발유,특별한렌트카,만 26세,완전자차,리뷰 63,누적예약 849,4.3,20210801,"99,700원",3
2,더 뉴 스파크(휘),경형,5인승,휘발유,제주e렌트카,만 23세,완전자차,리뷰 41,누적예약 555,4.3,20210801,"107,700원",2
3,올 뉴 모닝(휘),경형,5인승,휘발유,제주아산렌트카,만 23세,완전자차,리뷰 121,누적예약 844,4.2,20210801,"111,600원",1
4,더 뉴 레이(휘),경형,5인승,휘발유,제주e렌트카,만 23세,완전자차,리뷰 63,누적예약 992,4.1,20210801,"115,200원",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,볼보 S90 (휘),중형,5인승,휘발유,퍼스트렌트카,만 26세,완전자차,리뷰 0,누적예약 5,0.0,20210830,"495,600원",1
3638,BMW 520D(경),중형,5인승,경유,퍼스트렌트카,만 26세,완전자차,리뷰 3,누적예약 51,5.0,20210830,"495,600원",1
3639,테슬라모델3 EV(전기/충전비유료),전기,5인승,전기차,퍼스트렌트카,만 26세,완전자차,리뷰 6,누적예약 200,3.8,20210830,"495,600원",1
3640,BMW 430i 컨버터블(휘),중형,4인승,휘발유,퍼스트렌트카,만 26세,완전자차,리뷰 2,누적예약 20,5.0,20210830,"531,000원",1


In [8]:
rent.to_csv("august_rentcar.csv")
df_option.to_csv("august_options.csv")

In [ ]:
#concat